In [1]:
import sys
fileDir = "/home/jovyan/notebooks/"
sys.path.append(fileDir)

from utilities import *

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
session = create_spark_session("Tigergraph GitHub", SparkConnector.NEO4J)

Added dependencies: 
 ['neo4j-connector-apache-spark_2.12-5.0.1_for_spark_3.jar']


In [3]:
# Scenario 1

options = get_default_options(SparkConnector.NEO4J)
options["query"] =  """
                        MATCH (c:GitContributor)-[:AUTHOR]->(commit:GitCommit)-[:BELONGS_TO]->(r:GitRepository)
                        WITH c, COUNT(DISTINCT r) as repo_count
                        RETURN c, repo_count ORDER BY repo_count DESC
                    """
top10contributors = spark_read(SparkConnector.NEO4J, session, options=options)
display(top10contributors.take(10))

Dataframe loaded from neo4j


[Row(c=Row(<id>=11221, <labels>=['GitContributor'], name='Ikko Ashimine', email='d41f8067726d843438db002d5555099b4901d7c1@gmail.com'), repo_count=4),
 Row(c=Row(<id>=1721, <labels>=['GitContributor'], name='dependabot[bot]', email='1c358da00a777d4e9898c1280ab801e2df165188@users.noreply.github.com'), repo_count=3),
 Row(c=Row(<id>=4323, <labels>=['GitContributor'], name='Prayag Verma', email='35a46e17bc00e93336a001ea5a30f33595fd0d03@gmail.com'), repo_count=3),
 Row(c=Row(<id>=7027, <labels>=['GitContributor'], name='Michaël De Boey', email='59bd0a3ff43b32849b319e645d4798d8a5d1e889@michaeldeboey.be'), repo_count=3),
 Row(c=Row(<id>=4140, <labels>=['GitContributor'], name='James George', email='3e5d4505bd1e679d62cbd9e85b63ce0b6e249349@gmail.com'), repo_count=3),
 Row(c=Row(<id>=1097, <labels>=['GitContributor'], name='Ronald Eddy Jr', email='10bf4b03df8e0eeff31b9303c48728d238ba68d1@yahoo.com'), repo_count=3),
 Row(c=Row(<id>=4618, <labels>=['GitContributor'], name='Kohei TAKATA', email='3

In [11]:
# Scenario 2:
# Da completare e filtrare per javascript
LANGUAGE = "C++"
PERCENTAGE = 0.5
options = get_default_options(SparkConnector.NEO4J)
options["query"] = f"""
                    MATCH (r:GitRepository)-[w:WRITTED_IN]->(l:GitLanguage)
                    WITH r, SUM(w.bytes) AS totalBytesForRepo, COLLECT({{language_name: l.name,bytes: w.bytes}}) AS bytesForLanguages
                    UNWIND bytesForLanguages as bytesForLanguage
                    WITH r.name as repo_name, bytesForLanguage.language_name as lang, round((bytesForLanguage.bytes*1.0/totalBytesForRepo),2) as percOfBytes
                        WHERE lang = "{LANGUAGE}" AND percOfBytes > {PERCENTAGE}
                    RETURN repo_name, lang, percOfBytes 

                  """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
display(bytesPercentageInRepos.take(10))

Dataframe loaded from neo4j


[Row(repo_name='tensorflow/tensorflow', lang='C++', percOfBytes=0.63)]

In [10]:
# Scenario 3:
REPO_NAME = "tensorflow/tensorflow"
options = get_default_options(SparkConnector.NEO4J)
options["query"] =  f"""
                        MATCH (repository:GitRepository {{name: "{REPO_NAME}"}})<-[:BELONGS_TO]-(commit:GitCommit), 
                            r = (commit)-[:PARENT]->()
                        WITH commit, collect(r) AS parents
                            WHERE SIZE(parents) > 1
                        RETURN COUNT(commit) as mergeCount
                    """
bytesPercentageInRepos = spark_read(SparkConnector.NEO4J, session, options=options)
bytesPercentageInRepos.show(10)

Dataframe loaded from neo4j
+----------+
|mergeCount|
+----------+
|     12127|
+----------+



In [3]:
session.sparkContext.stop()
session.stop()